# _Influencia de la duración de piststops en carreras de F1_

* Hipótesis:  

- Mejor media de tiempos por carrera y temporada aumenta las posiciones en carrera y al final de temporada  

- Idem para constructores  

- Diferencias entre conductores de una mismo equipo

<img src="F1_logo.jpg"  width="200"> <p>
<img src="pitStopAstonMartin.jpg"  width="600" height="300">

### <span style="color:RED"> DATASET</span>

#### CSV's EXTRAIDOS DE ergast.com  

* Ofrece resultados de todas las carreras desde 1950 hasta hoy.  

* Paquete ergast_py  
https://pypi.org/project/ergast-py/

* API con datos actualizados tras cada carrera  
  
  * API documentación http://ergast.com/mrd/  
    
* Libreria propia para acceder a la API  
    pyERGAST


#### Ofrecen todo el paquete de csv hasta la carrera más reciente  
* f1db_csv.zip  
http://ergast.com/downloads/f1db_csv.zip  

* Diagrama de la BD  
http://ergast.com/images/ergast_db.png  

* Esquema de la BD  
http://ergast.com/docs/f1db_user_guide.txt  

Cada csv tiene una sola base de datos en UTF-8  
La primera línea son las cabeceras de las columnas. 




In [22]:
import pandas as pd
import numpy as np
import sqlite3
import os
import re

* 13 csv

In [3]:
circuits=pd.read_csv(r'./circuits.csv')
constructor_results=pd.read_csv(r'./constructor_results.csv')
constructor_standings=pd.read_csv(r'./constructor_standings.csv')
driver_standings=pd.read_csv(r'./driver_standings.csv')
drivers=pd.read_csv (r'./drivers.csv')
lap_times=pd.read_csv(r'./lap_times.csv')
pit_stops=pd.read_csv(r'./pit_stops.csv')
qualifying=pd.read_csv(r'./qualifying.csv')
races=pd.read_csv(r'./races.csv')
results=pd.read_csv(r'./results.csv')
seasons=pd.read_csv(r'./seasons.csv')
sprint_results=pd.read_csv(r'./sprint_results.csv')
status=pd.read_csv(r'./status.csv')


In [23]:
status[status.status.str.contains("[Tt]yre")]
# usar regex para filtrar valores de columnas

,statusId,status
26,27,Tyre
58,59,Tyre puncture


In [36]:
status.query('status.str.contains("[Tt]yre")')

,statusId,status
26,27,Tyre
58,59,Tyre puncture


* Para el primer proyecto EDA, los más interesantes son:  

    * resultados                    | *results.csv*  
    * conductores                   |  *drivers.csv*  
    * carreras                      |  *races.csv*  
    * resultados de constructores por campeonato   | *constructor_standings*  
    * resultados de constructores por carrera   | *constructor_results*  
    * pit stops                     | *pit_stops*  
    * status (finalizado/otros)     | *status*


In [18]:
# Resultados de cada carrera, identificadores de carrera, piloto, posición, puntos, vueltas, tiempos

results.sample(5)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
7466,7467,334,78,27,25,20,7,7,7,0.0,72,\N,\N,\N,\N,\N,\N,11
5899,5900,285,100,33,20,20,6,6,6,1.0,52,\N,\N,\N,\N,\N,\N,11
23369,23374,968,832,5,55,21,\N,R,18,0.0,41,\N,\N,30,18,1:46.591,187.580,130
4347,4348,222,64,21,2,16,12,12,12,0.0,52,\N,\N,\N,\N,\N,\N,11
21218,21221,859,22,3,11,12,14,14,14,0.0,70,\N,\N,70,6,1:16.684,202.289,11


In [16]:
# Información básica de cada conductor, nombre, apellido, id, alias

drivers.head(2)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld


In [17]:
# Info básica de cada carrera de 1950-2023, id, nombre

races.sample(5)

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
1002,1015,2019,6,6,Monaco Grand Prix,2019-05-26,13:10:00,http://en.wikipedia.org/wiki/2019_Monaco_Grand...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
340,341,2010,5,4,Spanish Grand Prix,2010-05-09,12:00:00,http://en.wikipedia.org/wiki/2010_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
818,819,1952,3,13,Belgian Grand Prix,1952-06-22,\N,http://en.wikipedia.org/wiki/1952_Belgian_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
520,521,1979,10,10,German Grand Prix,1979-07-29,\N,http://en.wikipedia.org/wiki/1979_German_Grand...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
334,335,1990,15,22,Japanese Grand Prix,1990-10-21,\N,http://en.wikipedia.org/wiki/1990_Japanese_Gra...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [38]:
# Resultado final por carrera por constructor, con puntos

constructor_results.sample(5)

,constructorResultsId,raceId,constructorId,points,status
9086,12835,585,74,0.0,\N
2990,2991,288,1,10.0,\N
5889,5890,476,57,0.0,\N
11071,15569,981,1,0.0,\N
10471,14970,914,15,0.0,\N


In [39]:
# Resultados por constructor por carrera, puntos, posición

constructor_standings.sample(5)

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
1976,5270,209,22,11.0,4,4,0
11253,26117,916,9,363.0,2,2,3
11517,26486,952,5,26.0,5,5,0
301,25873,48,9,16.0,5,5,0
8004,22067,609,63,0.0,11,11,0


In [41]:
# Paradas en boxes de cada carrera y conductor, con duración en segundos y milisegundos

pit_stops.sample(5)

,raceId,driverId,stop,lap,time,duration,milliseconds
7800,1041,154,1,28,14:57:54,23.635,23635
8768,1072,815,2,13,20:58:46,16:42.619,1002619
8472,1064,849,1,23,15:33:19,20.646,20646
6454,996,20,2,40,17:22:38,31.029,31029
5813,976,807,1,13,17:28:32,21.263,21263


In [6]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25980 entries, 0 to 25979
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         25980 non-null  int64  
 1   raceId           25980 non-null  int64  
 2   driverId         25980 non-null  int64  
 3   constructorId    25980 non-null  int64  
 4   number           25980 non-null  object 
 5   grid             25980 non-null  int64  
 6   position         25980 non-null  object 
 7   positionText     25980 non-null  object 
 8   positionOrder    25980 non-null  int64  
 9   points           25980 non-null  float64
 10  laps             25980 non-null  int64  
 11  time             25980 non-null  object 
 12  milliseconds     25980 non-null  object 
 13  fastestLap       25980 non-null  object 
 14  rank             25980 non-null  object 
 15  fastestLapTime   25980 non-null  object 
 16  fastestLapSpeed  25980 non-null  object 
 17  statusId    

In [8]:
# CARRERAS DONDE ALONSO HA SIDO 1º 2º o 3º
alonso_1_2_3=results[["raceId","driverId", "position"]].groupby("driverId").get_group(4)[(results.position=="1")|(results.position=="2")|(results.position=="3")]
alonso_1_2_3

C:\Users\j\AppData\Local\Temp\ipykernel_26660\154912858.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  alonso_1_2_3=results[["raceId","driverId", "position"]].groupby("driverId").get_group(4)[(results.position=="1")|(results.position=="2")|(results.position=="3")]


,raceId,driverId,position
288,32,4,1
308,33,4,1
349,35,4,2
369,36,4,2
390,37,4,1
...,...,...,...
25842,1098,4,3
25862,1099,4,3
25882,1100,4,3
25922,1102,4,3


In [56]:
results.groupby("driverId").size()

driverId
1      317
2      184
3      206
4      365
5      112
      ... 
854     44
855     29
856      8
857      7
858      7
Length: 857, dtype: int64